# [How to cache chat model responses](https://python.langchain.com/v0.2/docs/how_to/chat_model_caching/)

LangChain provides an optional caching layer for chat models. This is useful for two main reasons:

* It can save you money by reducing the number of API calls you make to the LLM provider, if you're often requesting the same completion multiple times. This is especially useful during app development.
* It can speed up your application by reducing the number of API calls you make to the LLM provider.

This guide will walk you through how to enable this in your apps.

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")



In [ ]:
# <!-- ruff: noqa: F821 -->
from langchain_core.globals import set_llm_cache

### In Memory Cache
This is an ephemeral cache that stores model calls in memory. It will be wiped when your environment restarts, and is not shared across processes.

In [ ]:
%%time

from langchain_core.caches import InMemoryCache

set_llm_cache(InMemoryCache())

# The first time, it is not yet in cache, so it should take longer
llm.invoke("Tell me a joke")

In [ ]:
%%time
# The second time it is, so it goes faster
llm.invoke("Tell me a joke")

## SQLite Cache
This cache implementation uses a SQLite database to store responses, and will last across process restarts.

In [ ]:
!rm .langchain.db

In [ ]:
# We can do the same thing with a SQLite cache
from langchain_community.cache import SQLiteCache

set_llm_cache(SQLiteCache(database_path=".langchain.db"))

In [ ]:
%%time
# The first time, it is not yet in cache, so it should take longer
llm.invoke("Tell me a joke")

In [ ]:
%%time
# The second time it is, so it goes faster
llm.invoke("Tell me a joke")